In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [5]:
from treemort.utils.config import setup

config_file_path = "../configs/flair_unet_bs8_cs256.txt"
conf = setup(config_file_path)

# Modified Config Variables for Local Execution; comment on HPC
conf.data_folder = "/Users/anisr/Documents/USA2Finland/model55_350"
conf.output_dir = os.path.join("..", conf.output_dir)
conf.hdf5_file = 'real_A.h5'
conf.resume = False

print(conf)

2024-10-18 14:48:29,201 - INFO - Using config file: ../configs/flair_unet_bs8_cs256.txt


2024-10-18 14:48:29,209 - INFO - Configuration successfully loaded.


Namespace(model='flair_unet', backbone=None, model_weights='best', learning_rate=0.0001, activation='sigmoid', loss='hybrid', threshold=0.5, class_weights=[0.1, 0.9], epochs=100, train_batch_size=8, val_batch_size=8, test_batch_size=8, train_crop_size=256, val_crop_size=256, test_crop_size=256, val_size=0.0, test_size=1.0, resume=False, data_folder='/Users/anisr/Documents/USA2Finland/model55_350', hdf5_file='real_A.h5', input_channels=4, output_channels=1, output_dir='../output/flair_unet')


In [116]:
from treemort.main import run

eval_only = True

run(conf, eval_only)

2024-10-18 10:24:54,964 - INFO - Output directory already exists: ../output/flair_unet
2024-10-18 10:24:54,965 - INFO - Using device: cpu
2024-10-18 10:24:54,966 - INFO - Preparing datasets...
2024-10-18 10:24:55,281 - INFO - Datasets prepared: Train(0), Val(0), Test(140)
2024-10-18 10:24:55,281 - INFO - Loading or resuming model...
2024-10-18 10:24:55,282 - INFO - Building model...
2024-10-18 10:24:56,113 - INFO - flair_unet model configured.
2024-10-18 10:24:56,116 - INFO - Model successfully moved to cpu.
2024-10-18 10:24:56,117 - INFO - Adam optimizer configured with learning rate 0.0001.
2024-10-18 10:24:56,117 - INFO - Hybrid loss and metrics (IOU, F-Score) configured.
2024-10-18 10:24:56,205 - INFO - Loaded weights from ../output/flair_unet/best.weights.pth.
2024-10-18 10:24:56,206 - INFO - Model, optimizer, criterion, metrics, and callbacks are set up.
2024-10-18 10:24:56,206 - INFO - Evaluation-only mode started.
Evaluating: 100%|██████████| 140/140 [02:15<00:00,  1.03it/s, it

### Optional. Plot samples

In [6]:
import torch

from treemort.data.loader import prepare_datasets
from treemort.modeling.builder import resume_or_load
from treemort.evaluation.evaluator import evaluator

id2label = {0: "alive", 1: "dead"}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

conf.hdf5_file = 'real_A.h5'
_, _, test_dataset_real = prepare_datasets(conf)

conf.hdf5_file = 'fake_B.h5'
_, _, test_dataset_fake = prepare_datasets(conf)

model, _, _, _, _ = resume_or_load(conf, id2label, len(test_dataset_fake), device)

2024-10-18 14:48:32,512 - INFO - Building model...
2024-10-18 14:48:33,446 - INFO - flair_unet model configured.
2024-10-18 14:48:33,449 - INFO - Model successfully moved to cpu.
2024-10-18 14:48:33,450 - INFO - Adam optimizer configured with learning rate 0.0001.
2024-10-18 14:48:33,451 - INFO - Hybrid loss and metrics (IOU, F-Score) configured.
2024-10-18 14:48:33,451 - INFO - Training model from scratch.


In [9]:
import torch
import matplotlib.pyplot as plt

# Switch the model to evaluation mode
model.eval()

# Get one sample from the test_dataset_fake
images_fake, labels_fake, keys_fake = next(iter(test_dataset_fake))  # Assume keys_fake is available
sample_index = 0  # You can change this index to retrieve different samples

# Find the corresponding key for the sample
sample_key = keys_fake[sample_index]  # The unique identifier (could be filename or another key)

# Use this key to find the corresponding sample in the test_dataset_real
for images_real, labels_real, keys_real in test_dataset_real:
    if sample_key in keys_real:
        index_real = keys_real.index(sample_key)  # Get the index for the same sample
        image_real = images_real[index_real]
        label_real = labels_real[index_real]
        break

# Now retrieve the fake sample with the same key
image_fake = images_fake[sample_index]
label_fake = labels_fake[sample_index]

# No gradients needed during evaluation
with torch.no_grad():
    # Get predictions for both real and fake images
    outputs_fake = model(image_fake.unsqueeze(0))  # Add batch dimension
    predictions_fake = torch.sigmoid(outputs_fake).squeeze(0)
    
    outputs_real = model(image_real.unsqueeze(0))  # Add batch dimension
    predictions_real = torch.sigmoid(outputs_real).squeeze(0)

# Convert tensors to numpy arrays for plotting
rgb_image_fake = image_fake[1:, :, :].permute(1, 2, 0).cpu().numpy()
label_image_fake = label_fake.squeeze().cpu().numpy()
prediction_image_fake = predictions_fake.squeeze().cpu().numpy()
prediction_mask_fake = prediction_image_fake > conf.threshold

rgb_image_real = image_real[1:, :, :].permute(1, 2, 0).cpu().numpy()
label_image_real = label_real.squeeze().cpu().numpy()
prediction_image_real = predictions_real.squeeze().cpu().numpy()
prediction_mask_real = prediction_image_real > conf.threshold

# Plot the images and their predictions side by side for real and fake datasets
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Fake dataset visualizations
axes[0, 0].imshow(rgb_image_fake)
axes[0, 0].set_title("Fake Dataset - NIR+RGB Image (RGB view)")
axes[0, 0].axis('off')

axes[0, 1].imshow(label_image_fake, cmap='gray')
axes[0, 1].set_title("Fake Dataset - Label (Grayscale)")
axes[0, 1].axis('off')

axes[0, 2].imshow(prediction_image_fake, cmap='gray')
axes[0, 2].set_title("Fake Dataset - Prediction (Grayscale)")
axes[0, 2].axis('off')

axes[0, 3].imshow(prediction_mask_fake, cmap='gray')
axes[0, 3].set_title("Fake Dataset - Prediction (Binary)")
axes[0, 3].axis('off')

# Real dataset visualizations
axes[1, 0].imshow(rgb_image_real)
axes[1, 0].set_title("Real Dataset - NIR+RGB Image (RGB view)")
axes[1, 0].axis('off')

axes[1, 1].imshow(label_image_real, cmap='gray')
axes[1, 1].set_title("Real Dataset - Label (Grayscale)")
axes[1, 1].axis('off')

axes[1, 2].imshow(prediction_image_real, cmap='gray')
axes[1, 2].set_title("Real Dataset - Prediction (Grayscale)")
axes[1, 2].axis('off')

axes[1, 3].imshow(prediction_mask_real, cmap='gray')
axes[1, 3].set_title("Real Dataset - Prediction (Binary)")
axes[1, 3].axis('off')

plt.tight_layout()
plt.show()

tensor([468, 619, 461, 445, 925, 252, 634, 662])
tensor([790, 274, 692, 116, 253, 969, 912, 658])
tensor([1059,  641,  146,  191,  919,  718,  457,  767])
tensor([ 70, 607, 413, 529, 736, 654, 503, 620])
tensor([ 168, 1045,  781,  589, 1044,  390,  778,  186])
tensor([ 470, 1061,   80,  573,  326,  548, 1085,  482])
tensor([ 417,   47,  590,  971,  463,  248, 1117,  105])
tensor([851, 549, 311, 852, 241, 667, 876, 511])
tensor([ 597,  621,  332,  172, 1010,   64,  877,  792])
tensor([937, 388, 212, 505, 976, 904, 130,  75])
tensor([ 729,  293,  707, 1096,  133,  122,  681,  287])
tensor([902, 497, 415, 695, 568, 128, 606, 715])
tensor([1023,  668,  618,  861,  835,  433,  336,  475])
tensor([ 483,  717,  992,  451,  224,  440,   82, 1017])
tensor([578, 727, 366, 917, 396,  41, 834, 967])
tensor([1041,  614,  832,  866, 1003,  385,  633,  151])
tensor([ 955,  200,  637,  138,  586, 1103,  819,  198])
tensor([1014,  639,  743,  435, 1033,  624,  653,  789])
tensor([156, 132, 230, 609, 48

AttributeError: 'Tensor' object has no attribute 'index'